# Hotel Recommendation System(ML Assignment)


# Motivation
<html>
    <body>
This is a part of my internship experience which I want to share <br>
how I started building a hotel recommender system for managers <br>
who needs an insight about how their hotel is ranked compared <br>
with other hotels from scratch.
    </body>
</html>

# Description of Complete Problem

<html>
    <body>
        <br>
        Our aim is to help managers from hotels to find how their hotels are ranked.<br>
        In the middle eastern countries there is a concept called dynamic pricing.<br>
        The rate of pricing in hotels change from season to season and based on demand<br>
        Our motive is to give them an estimate to help the officials fix the price based on<br> the rank and meta data we supply.
    </body>
 </html>

# Data Retrieval

There are several packages in Python that allow us to scrape information from webpages.<br>
One of the most common ones is BeautifulSoup.<br>
BeautifulSoup allows us to parse the HTML content of a given URL and<br> 
access its elements by identifying them with their tags and attributes.<br> 
For this reason, we will use it to extract certain pieces of text from the websites.<br>
It is an extremely easy-to-use yet powerful package. <br>
With almost 3–5 lines of code we will be able to extract any text we want from the internet.<br>
<br>
alternate tools like scrapy,diffbot,parseHub also are famous 
<br><br> 
**For our work we have used a combination of Requests and Beautiful Soup**<br>
We have scraped the data from 9 different websites


# Data Description

<br>
The data contains infomations like basic name,adress,location,ratings<br>
In another file we have collected the corresponding hotel details and<br>
information from their url and stored it which contains room amenities,roomtypes<br>
and features the hotel had.<br>
There is one seperate file which is collected from booking websites(trivago) <br>
which holds it pricing information.<br>


# Approach Taken(Algorithm)

<br>
<p>
We have gone with Case Based Reasoning (CBR) approach to analyse the data.
Case-based reasoning (CBR), broadly construed, is the process of solving new problems based on the 
solutions of similar past problems.An auto mechanic who fixes an engine by recalling another car that
exhibited similar symptoms is using case-based reasoning. A lawyer who advocates a particular outcome
in a trial based on legal precedents or a judge who creates case law is using case-based reasoning.
So, too, an engineer copying working elements of nature (practicing biomimicry),
is treating nature as a database of solutions to problems.
Case-based reasoning is a prominent type of analogy solution making.</p>
<p>
It has been argued that case-based reasoning is not only a powerful method for computer reasoning, but also a pervasive behavior in everyday human problem solving; or, more radically, that all reasoning is based on past cases personally experienced. This view is related to prototype theory, which is most deeply explored in cognitive science.</p>


**Process**
<p>
    Retrieve: Given a target problem, retrieve from memory cases relevant to solving it. A case consists of a problem, its solution, and, typically, annotations about how the solution was derived. For example, suppose Fred wants to prepare blueberry pancakes. Being a novice cook, the most relevant experience he can recall is one in which he successfully made plain pancakes. The procedure he followed for making the plain pancakes, together with justifications for decisions made along the way, constitutes Fred's retrieved case.</p>
    <p>
Reuse: Map the solution from the previous case to the target problem. This may involve adapting the solution as needed to fit the new situation. In the pancake example, Fred must adapt his retrieved solution to include the addition of blueberries.
    </p>
    <p>
Revise: Having mapped the previous solution to the target situation, test the new solution in the real world (or a simulation) and, if necessary, revise. Suppose Fred adapted his pancake solution by adding blueberries to the batter. After mixing, he discovers that the batter has turned blue – an undesired effect. This suggests the following revision: delay the addition of blueberries until after the batter has been ladled into the pan.</p>
<p>
Retain: After the solution has been successfully adapted to the target problem, store the resulting experience as a new case in memory. Fred, accordingly, records his new-found procedure for making blueberry pancakes, thereby enriching his set of stored experiences, and better preparing him for future pancake-making demands.
</p>

# Disadvantages of CBR

<P>
Critics of CBR argue that it is an approach that accepts anecdotal evidence as its main operating principle. Without statistically relevant data for backing and implicit generalization, there is no guarantee that the generalization is correct. However, all inductive reasoning where data is too scarce for statistical relevance is inherently based on anecdotal evidence. There is recent work that develops CBR within a statistical framework and formalizes case-based inference as a specific type of probabilistic inference; thus, it becomes possible to produce case-based predictions equipped with a certain level of confidence. One description of the difference between CBR and induction from instances is that statistical inference aims to find what tends to make cases similar while CBR aims to encode what suffices to claim similarly.   </P>

# Targeted Audience for the application

This is primarly intended for hotel managers and stake holders to have a stand among online room bookers.<br>
However this application could also be extended to clients of an e-booking platform

# COD3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


# Fetching the data

In [ ]:
hotel_details=pd.read_csv('../input/hotel-recommendation/Hotel_details.csv',delimiter=',')
hotel_rooms=pd.read_csv('../input/hotel-recommendation/Hotel_Room_attributes.csv',delimiter=',')
hotel_cost=pd.read_csv('../input/hotel-recommendation/hotels_RoomPrice.csv',delimiter=',')

In [ ]:
hotel_details.head()

In [ ]:
hotel_rooms.head()

# Data Cleaning and transformations

In [ ]:
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']

In [ ]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()

In [ ]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

In [ ]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

In [ ]:
hotel.columns

In [ ]:
del hotel['hotelid']
del hotel['url']
del hotel['curr']
del hotel['Source']

In [ ]:
hotel.columns

# Recommender system based only on City and ratings about the hotel

In [ ]:
def citybased(city):
    hotel['city']=hotel['city'].str.lower()
    citybase=hotel[hotel['city']==city.lower()]
    citybase=citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

In [ ]:
print('Top 5 hotels')
citybased('London')

In [ ]:
room_no=[
     ('king',2),
   ('queen',2), 
    ('triple',3),
    ('master',3),
   ('family',4),
   ('murphy',2),
   ('quad',4),
   ('double-double',4),
   ('mini',2),
   ('studio',1),
    ('junior',2),
   ('apartment',4),
    ('double',2),
   ('twin',2),
   ('double-twin',4),
   ('single',1),
     ('diabled',1),
   ('accessible',1),
    ('suite',2),
    ('one',2)
   ]

In [ ]:
def calc():
    guests_no=[]
    for i in range(hotel.shape[0]):
        temp=hotel['roomtype'][i].lower().split()
        flag=0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j]==room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag=1
                    break
            if flag==1:
                break
        if flag==0:
            guests_no.append(2)
    hotel['guests_no']=guests_no

calc()

In [ ]:
def pop_citybased(city,number):
    hotel['city']=hotel['city'].str.lower()
    popbased=hotel[hotel['city']==city.lower()]
    popbased=popbased[popbased['guests_no']==number].sort_values(by='starrating',ascending=False)
    popbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if popbased.empty==True:
        print('Sorry No Hotels Available\n tune your constraints')
    else:
        return popbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription']].head(10)
    
    

In [ ]:
pop_citybased('London',4)

In [ ]:
hotel['roomamenities']=hotel['roomamenities'].str.replace(': ;',',')

In [ ]:
def requirementbased(city,number,features):
    hotel['city']=hotel['city'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['city']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)

# Requirement And special needs based Recommender

In [ ]:
requirementbased('London',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

In [ ]:
from math import sin, cos, sqrt, atan2, radians
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = '4519d076b432f5'
import requests
R=6373.0#Earth's Radius
sw = stopwords.words('english')
lemm = WordNetLemmatizer()
def hybrid(address,city,number,features):
    features=features.lower()
    features_tokens=word_tokenize(features)
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    data = {
    'key': secret_value_0,
    'q': address,
    'format': 'json'}
    response = requests.get(url, params=data)
    dist=[]
    lat1,long1=response.json()[0]['lat'],response.json()[0]['lon']
    lat1=radians(float(lat1))
    long1=radians(float(long1))
    hybridbase=hotel[hotel['guests_no']==number]
    hybridbase['city']=hybridbase['city'].str.lower()
    hybridbase=hybridbase[hybridbase['city']==city.lower()]
    hybridbase.drop_duplicates(subset='hotelcode',inplace=True,keep='first')
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    for i in range(hybridbase.shape[0]):
        lat2=radians(hybridbase['latitude'][i])
        long2=radians(hybridbase['longitude'][i])
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist.append(distance)
    hybridbase['distance']=dist
    hybridbase=hybridbase.sort_values(by='distance',ascending=True)
    hybridbase=hybridbase.head(15)
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    coss=[]
    for i in range(hybridbase.shape[0]):
        temp_tokens=word_tokenize(hybridbase['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        coss.append(len(rvector))
    hybridbase['similarity']=coss
    return hybridbase.sort_values(by='similarity',ascending=False).head(10)
    
    
        

In [ ]:
url = "https://us1.locationiq.com/v1/search.php"
hybrid("Big Ben,London",'London',4,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

In [ ]:
hotel_cost.head()
#onsite rate is one important feature which could be useful to recommend
#we will drop the rest since it is present in other table and we are going to merge the 
hotel_cost=hotel_cost.drop(['id','refid','websitecode','dtcollected','ratedate','los','guests','roomtype','netrate','ratedescription','ratetype','sourceurl','roomamenities'
,'ispromo','closed','discount','promoname','status_code','taxstatus','taxtype','taxamount','proxyused','israteperstay','hotelblock','input_dtcollected'],axis=1)

In [ ]:
hotel_cost.columns

In [ ]:
#To reccomend we are gonna check how much does the price vary from room to room if 
#the varience is small enough then it is better for them to recommend the hotel
hot=hotel_cost.groupby(['hotelcode','maxoccupancy'])

In [ ]:
hotel_cost.sort_values(by=['onsiterate'],ascending=False)
hotel_cost=hotel_cost.drop_duplicates(subset=['hotelcode','maxoccupancy'],keep='first')

In [ ]:
var=hot['onsiterate'].var().to_frame('varience')
l=[]
for i in range(hotel_cost.shape[0]):
    var1=var[var.index.get_level_values(0)==hotel_cost.iloc[i][0]]
    l.append(var1[var1.index.get_level_values(1)==hotel_cost.iloc[i][3]]['varience'][0])

In [ ]:
hotel_cost['var']=l
hotel_cost=hotel_cost.fillna(0)
hotel_cost['mealinclusiontype']=hotel_cost['mealinclusiontype'].replace(0,'No Complimentary')

In [ ]:
hotel1=pd.merge(hotel,hotel_cost,left_on=['hotelcode','guests_no'],right_on=['hotelcode','maxoccupancy'],how='inner')

In [ ]:
hotel1=hotel1.drop_duplicates(subset=['hotelcode','maxoccupancy'],keep='first')

In [ ]:
hotel1.columns

# Recommendder with pricing included

In [ ]:
def pricing(address,city,number,features):
    h=hybrid(address,city,number,features)
    price_based=pd.merge(h,hotel_cost,left_on=['hotelcode','guests_no'],right_on=['hotelcode','maxoccupancy'],how='inner')
    del price_based['maxoccupancy']
    h=price_based.sort_values(by='var')
    return h.head()

In [ ]:
pricing("Tower of London",'London',4,'I need an alarm clock and a kettle flask.')
#the description is just only for example.It portrays you can keep any specific request

# After applying Optimum band formula and simmilarity index

In [ ]:
optimum_band=pd.read_csv('../input/hotel-recommendation/hotel_price_min_max - Formula.csv')

In [ ]:
from math import sin, cos, sqrt, atan2, radians
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = '4519d076b432f5'
import requests
R=6373.0#Earth's Radius
sw = stopwords.words('english')
lemm = WordNetLemmatizer()
def hybrid(address,city,number,features):
    features=features.lower()
    features_tokens=word_tokenize(features)
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    data = {
    'key': secret_value_0,
    'q': address,
    'format': 'json'}
    response = requests.get(url, params=data)
    dist=[]
    lat1,long1=response.json()[0]['lat'],response.json()[0]['lon']
    lat1=radians(float(lat1))
    long1=radians(float(long1))
    hybridbase=hotel
    #hybridbase=hotel[hotel['guests_no']==number]
    hybridbase['city']=hybridbase['city'].str.lower()
    hybridbase=hybridbase[hybridbase['city']==city.lower()]
    hybridbase.drop_duplicates(subset='hotelcode',inplace=True,keep='first')
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    for i in range(hybridbase.shape[0]):
        lat2=radians(hybridbase['latitude'][i])
        long2=radians(hybridbase['longitude'][i])
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist.append(distance)
    hybridbase['distance']=dist
    hybridbase=hybridbase[hybridbase['distance']<=5]
    hybridbase=hybridbase.set_index(np.arange(hybridbase.shape[0]))
    coss=[]
    for i in range(hybridbase.shape[0]):
        temp_tokens=word_tokenize(hybridbase['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        coss.append(len(rvector))
    hybridbase['similarity']=coss
    return hybridbase.sort_values(by='similarity',ascending=False)

def price_band_based(address,city,number,features):
    h=hybrid(address,city,number,features)
    price_band=pd.merge(h,optimum_band,left_on=['hotelcode'],right_on=['hotelcode'],how='inner')
    price_band=pd.merge(price_band,hotel_cost,left_on=['hotelcode'],right_on=['hotelcode'],how='inner')
    del price_band['min']
    del price_band['max']
    del price_band['Diff_Min']
    del price_band['Diff_Max']
    del price_band['currency']
    del price_band['country']
    del price_band['propertytype']
    del price_band['starrating']
    del price_band['latitude']
    del price_band['longitude']
    del price_band['guests_no']
    del price_band['var']
    price_band=price_band[price_band['Score']<=0.5]
    price_band=price_band[price_band['maxoccupancy']>=number]
    return price_band

In [ ]:
price_band_based("Tower of London",'London',4,'I need an alarm clock and a kettle flask.').head(5)

# Conclusion

After all these a reccommender system from scratch is made which considers the following features while recommending<br>
1. Closness to location
2. Number of guests
3. Onsite Rate
4. Credibility of the hotel
5. Room Ammenities
6. Complimentary gifts(Like Free Meals,Take aways)
7. Optimum price band
8. Any special requests the customer needs(Using NLTK search engines)

Later it was converted as a flask web app and delivered to the stakeholders.

In [ ]:

import pickle
Pkl_Filename = "Pickle_RL_Model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(price_band_based, file)